In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
from PIL import Image
import json
import requests
import pickle

#url open to get image
import urllib.request
from urllib.request import urlopen

#get data in parallel
from multiprocessing import Pool

#plot (for image verification)
import matplotlib.cm as cm
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
path_data = os.path.join(ROOT_DIR,'datasets/wikipedia')

In [3]:
PACKAGE_PARENT = '../../..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from UTILS.utils import search_wikipedia_laguage

/home/camille/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
#Note:
#their is some redirection in wikipedia, for example 'Acanthophis antarcticus' was redirected to 'Common death adder'
#Indeed, if we avoid these redirection we will loose some info as in the previous example. so we let them, and one
#should later verify all the onw that has been redirected.

## download synonyms and species names

In [4]:
df_species_syn = pd.read_csv(os.path.join(ROOT_DIR, 'datasets/synonyms/df_species_syn_andrew.csv'), sep=',')
print(df_species_syn.shape)
df_species_syn.head(3)

(3730, 7)


,binomial,split2,split2_clean,li_synonyms,li_synonyms_clean,syn_equal_other_species,li_synonyms_final
0,Hydrophis peronii,"c(""Acalyptus Peronii"", ""? Acalyptus"", ""Acalypt...","['Pseudodisteira horrida', 'Acalyptophis peron...","['Pseudodisteira horrida', 'Acalyptophis peron...","['Pseudodisteira horrida', 'Acalyptophis peron...",[],"['Pseudodisteira horrida', 'Acalyptophis peron..."
1,Acanthophis antarcticus,"c(""Boa antarctica"", ""Acanthophis cerastinus"", ...","['Vipera acanthophis', 'Vipera sorda', 'Boa an...","['Vipera acanthophis', 'Vipera sorda', 'Boa an...","['Vipera acanthophis', 'Vipera sorda', 'Boa an...",[],"['Vipera acanthophis', 'Vipera sorda', 'Boa an..."
2,Acanthophis hawkei,"c(""Acanthophis hawkei"", ""Acanthophis cummingi""...","['Acanthophis cummingi', 'Acanthophis hawkei']",['Acanthophis cummingi'],['Acanthophis cummingi'],[],['Acanthophis cummingi']


In [5]:
#create a list of species
li_species = df_species_syn['binomial'].tolist()
if len(li_species)!=len(set(li_species)):
    print('EREUR non unique species name')
    sys.exit()
print('There is %d species'%len(li_species))

There is 3730 species


## search for wiki language

In [ ]:
df = pd.DataFrame()
for species in tqdm.tqdm(li_species):
    content = search_wikipedia_laguage(text=species,language='en')
    d_p = content['query']['pages']
    for p in d_p.keys(): 
        if 'langlinks' in d_p[p]:
            df_lang = pd.DataFrame(d_p[p]['langlinks'])
            df_lang = df_lang.filter(['langname', '*'])
            df_lang = df_lang.set_index('langname').T
            df_lang['pageid'] = p
            df_lang['title'] = d_p[p]['title']
            df_lang['species'] = species
            df = pd.concat([df,df_lang], ignore_index=True)

In [7]:
print(df.shape)
df.head(3)

(1757, 144)


,Afrikaans,Albanian,Alemannisch,Arabic,Aragonese,Armenian,Arpitan,Assamese,Asturian,Aymara,...,Waray,Welsh,Western Frisian,Western Mari,Western Punjabi,Xhosa,Zazaki,pageid,species,title
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Acalyptophis peronii,NaN,NaN,NaN,اکالیپٹوفس پیرونی,NaN,NaN,26760494,Hydrophis peronii,Hydrophis peronii
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Acanthophis antarcticus,NaN,NaN,NaN,موتوالا سپ,NaN,NaN,6166502,Acanthophis antarcticus,Common death adder
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Acanthophis hawkei,NaN,NaN,NaN,NaN,NaN,NaN,49485486,Acanthophis hawkei,Acanthophis hawkei


In [8]:
#save
df.to_csv(os.path.join(path_data,'df_species_language.csv'), index=False, sep=';')